<a href="https://colab.research.google.com/github/alssalo/Notebooks/blob/master/m5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
## **Data Load using Kaggle API**
```



In [2]:
! pip install -q kaggle
from google.colab import files
files.upload()

{}

In [4]:
#! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c m5-forecasting-accuracy
! cd /content

 63% 9.00M/14.2M [00:00<00:00, 88.5MB/s]
100% 14.2M/14.2M [00:00<00:00, 90.9MB/s]
  0% 0.00/163k [00:00<?, ?B/s]
100% 163k/163k [00:00<00:00, 159MB/s]
 58% 9.00M/15.5M [00:00<00:00, 35.9MB/s]
100% 15.5M/15.5M [00:00<00:00, 51.8MB/s]
  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 105MB/s]


In [5]:
ls

calendar.csv  sales_train_validation.csv.zip  sample_submission.csv.zip
kaggle.json   sample_data/                    sell_prices.csv.zip


In [0]:
!unzip -q sell_prices.csv.zip 
!unzip -q sample_submission.csv.zip
!unzip -q sales_train_validation.csv.zip

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max< np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
def create_df(is_train = True,start_day = 1):

  date_cols = [f"d_{day}" for day in range(start_day,1914)]
  catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']

  sales_train_validation = pd.read_csv("sales_train_validation.csv")
  sales_train_validation = reduce_mem_usage(sales_train_validation)

  calender = pd.read_csv("calendar.csv")
  calender = reduce_mem_usage(calender)

  sell_prices = pd.read_csv("sell_prices.csv")
  sell_prices = reduce_mem_usage(sell_prices)

  sample_submission = pd.read_csv("sample_submission.csv")



  df = pd.melt(sales_train_validation,id_vars =catcols,
               value_vars = date_cols,
                var_name = "d",
                value_name = "sales")
  
  df = df.merge(calender , on = "d" , copy=False)
  df = df.merge(sell_prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

  return(df,sample_submission)

In [14]:
%%time
df,sample_submission = create_df() 

Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)
CPU times: user 2min 46s, sys: 9.89 s, total: 2min 56s
Wall time: 2min 56s


In [0]:
def normalize(df):
  scaler = MinMaxScaler()
  df = scaler.fit_transform(df)
  return df

def train_test_split(df):
  train_data = df[]
  test_data = df[]
  return(train_data,test_data)
  
def make_window(df):
  Xdata ,Ydata = [] ,[]
    for i in range (len(dataset)-inp-1):
        a = dataset[i:(i+inp),0]
        Xdata.append(a)
        Ydata.append(dataset[i+inp,0])
    return  np.array(Xdata), np.array(Ydata) 


def preprocess():

  #normalize data
  scaler = MinMaxScaler()
  df = scaler.fit_transform(df)

  #test-train split
  train_data ,test_data = train_test_split(df)

  #create rolling window
  Xtrain,ytrain = make_window(train_data)
  Xtest,ytest  = make_window,test_data))


In [0]:
Xtrain , ytrain , Xtest ,ytest = preproces(df)

In [0]:
df_sorted = df.sort_values(by=["id","d"]).reset_index(drop=True)

In [17]:
scaler = MinMaxScaler()
scaler.fit_transform(df)

ValueError: ignored